In [130]:
import pandas as pd
import numpy as np
import os

In [131]:
with open("sent_corpus.csv", "r") as sent_file:
    lines = sent_file.read().split("\n")

In [111]:
%time rows = [line.split(",") for line in lines if line]
%time rows = [row[:3] + [",".join(row[3:])] for row in rows]
# remove document start character 
rows[0][0] = rows[0][0][1:]

sentDf_cols = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']


%time sentDf_a = pd.DataFrame(rows[1:],columns=sentDf_cols)

#print(sentDf)

sentDf_a[["ItemID","Sentiment"]] = sentDf_a[["ItemID","Sentiment"]].astype(int)
%time sentDf_a["SentimentText"] = sentDf_a["SentimentText"].apply(lambda text: text.split())

CPU times: user 3.25 s, sys: 1.89 s, total: 5.14 s
Wall time: 6.33 s
CPU times: user 4.48 s, sys: 157 ms, total: 4.64 s
Wall time: 4.65 s
CPU times: user 377 ms, sys: 52.4 ms, total: 429 ms
Wall time: 475 ms
CPU times: user 4.82 s, sys: 834 ms, total: 5.65 s
Wall time: 5.89 s


In [132]:
import gensim
%time w2vM = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
dim = 300



CPU times: user 1min 29s, sys: 29.5 s, total: 1min 58s
Wall time: 2min 28s


In [97]:
%time w2vM = gensim.models.Word2Vec(sentDf["SentimentText"])
dim = 100

CPU times: user 1min 21s, sys: 2.53 s, total: 1min 24s
Wall time: 38.5 s


In [146]:
path = os.getcwd()+"/fullData/"
with open(path+"us_train.labels", "r") as labels_file:
    lines_test = labels_file.read().split("\n")
    
with open(path+"us_trial.labels", "r") as labels_file:
    lines_trial = labels_file.read().split("\n")
    
labels_train = pd.DataFrame(lines_test[:], columns=['SentimentLabels'])
labels_test = pd.DataFrame(lines_trial[:], columns=['SentimentLabels'])
print(labels_test)

      SentimentLabels
0                   0
1                   7
2                  11
3                   0
4                   2
5                   1
6                   8
7                   0
8                  13
9                   9
10                  9
11                 16
12                  1
13                 11
14                  0
15                  0
16                 19
17                  0
18                  1
19                  7
20                 19
21                  1
22                  4
23                  6
24                 14
25                  2
26                  2
27                  2
28                 13
29                  5
...               ...
49970               6
49971               1
49972               2
49973               6
49974               0
49975               1
49976               1
49977               0
49978               0
49979               1
49980               4
49981               2
49982              12
49983     

In [164]:
path = os.getcwd()+"/fullData/"
with open(path+"us_train.text", "r") as tweet_text_file:
    lines = tweet_text_file.read().split("\n")
    

    
sentDf_train = pd.DataFrame(lines[:], columns=['SentimentText'])
%time sentDf_train["SentimentText"] = sentDf_train["SentimentText"].apply(lambda text: text.split())
#print(sentDf[1:])
    
with open(path+"us_trial.text", "r") as tweet_text_file:
    lines = tweet_text_file.read().split("\n")

    
sentDf_test = pd.DataFrame(lines[:], columns=['SentimentText'])
%time sentDf_test["SentimentText"] = sentDf_test["SentimentText"].apply(lambda text: text.split())
#print(sentDf[1:])

CPU times: user 3.63 s, sys: 6.14 s, total: 9.78 s
Wall time: 17.1 s
CPU times: user 80.6 ms, sys: 12 ms, total: 92.6 ms
Wall time: 90 ms


In [165]:
# number of samples to aggregate
N_train = int(1e5)
N_test = labels_test.size

In [166]:
%%time
# MEAN AGGREGATION
tvecs_train = np.array([np.array([w2vM[t] if t in w2vM
                                else np.zeros((dim,))
                            for t in twt]).mean(axis=0)
                 for twt in sentDf_train['SentimentText'][:N_train]])

tvecs_test = np.array([np.array([w2vM[t] if t in w2vM
                                else np.zeros((dim,))
                            for t in twt]).mean(axis=0)
                 for twt in sentDf_test['SentimentText'][:N_test]])


CPU times: user 12.2 s, sys: 1.28 s, total: 13.5 s
Wall time: 18.1 s


In [168]:
X_train = tvecs_train[:N]
X_test = tvecs_test[:N]

print(labels_train['SentimentLabels'][:N_train].values)
print(labels_test['SentimentLabels'][:N_test].values)

y_pre_train = labels_train['SentimentLabels'][:N_train].values
y_pre_test = labels_test['SentimentLabels'][:N_test].values

y_train = np.array(y_pre_train).astype(np.int)
y_test = np.array(y_pre_test).astype(np.int)

print(y_test)
print(X)

['2' '17' '0' ..., '9' '9' '6']
['0' '7' '11' ..., '11' '0' '9']
[ 0  7 11 ..., 11  0  9]
[[-0.03525391  0.06650391  0.06716309 ..., -0.05820313 -0.02873535
   0.03405762]
 [-0.00695801  0.06316584 -0.05490945 ..., -0.06491921  0.00937722
  -0.03728693]
 [-0.03508301 -0.05007324 -0.04908447 ..., -0.0609375  -0.00522461
   0.06296082]
 ..., 
 [ 0.02618408  0.02658081  0.0098877  ..., -0.05737305  0.02505493
   0.06640625]
 [-0.00427246  0.09061686  0.02342394 ..., -0.0339951   0.05254449
   0.02824571]
 [ 0.00064087  0.01641846  0.03424072 ..., -0.02520752  0.01345825
   0.00135803]]


In [169]:
# generate test/train split
ratio = 0.8
tidx = np.random.rand(N) < ratio
pidx = ~tidx

In [170]:
import sklearn
from sklearn import ensemble,svm,neural_network,discriminant_analysis
from sklearn.metrics import roc_curve,auc,precision_recall_curve

from matplotlib import pyplot as plt
% matplotlib inline

In [203]:
rf = sklearn.ensemble.RandomForestClassifier()
rf.max_depth = 7

%time rf.fit(X_train, y_train)

CPU times: user 12.4 s, sys: 137 ms, total: 12.5 s
Wall time: 13.1 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
num_nodes = 140
hidden_layer_config = (num_nodes,)

mlp = sklearn.neural_network.MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes= hidden_layer_config, learning_rate='constant',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


#fit to training data
#%time mlp.fit(X[tidx],y_train[tidx])
%time mlp.fit(X_train, y_train)

#51% is baseline

In [ ]:
def getAccuracy(model, X_train, X_test, y_train, y_test):
    y_pred_pre_train = model.predict(X_train)
    y_pred_train = np.array(y_pred_pre_train).astype(np.int)

    y_pred_pre_test = model.predict(X_test)
    y_pred_test = np.array(y_pred_pre_test).astype(np.int)

    num_same_train = 0
    num_same_test = 0

    for i in y:
        if(y_pred_train[i] == y_train[i]):
            num_same_train += 1
        if(y_pred_test[i] == y_test[i]):
            num_same_test += 1

#    print(num_same_train)    
 #   print(num_same_test)

    accuracy_train = float(num_same_train) / y_train.size
    accuracy_test = float(num_same_test) / y_test.size
    
    return accuracy_train, accuracy_test

In [ ]:
from sklearn.externals import joblib

import pickle
print("nn stats... ")
nn_acc_train, nn_acc_test = getAccuracy(mlp, X_train, X_test, y_train, y_test)
print("training accuracy: " + str(nn_acc_train))
print("test accuracy: " + str(nn_acc_test))
if(nn_acc_test > 0.4):
    print("Good model. Saving...")
    filename_acc = str(nn_acc_test)[2:4]
    filename = "model_acc"+filename_acc+"hl"+str(num_nodes)+.pkl"
    joblib.dump(mlp, filename)
#    pickle.dump(mlp, open(filename, 'wb'))
    
print("model saved")

#print("rf stats... ")
#rf_acc_train, rf_acc_test = getAccuracy(rf, X_train, X_test, y_train, y_test)
#print("training accuracy: " + str(rf_acc_train))
#print("test accuracy: " + str(rf_acc_test))

#140 hidden layer
#training accuracy: 0.33859
#test accuracy: 0.69872

In [77]:
n_classes = 20


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:i], y_pred[:i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

ValueError: Data is not binary and pos_label is not specified

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()